In [1]:
import pandas as pd
import numpy as np

In [2]:
from telegram_notifier import send_message as telegram_bot_sendtext

In [3]:
import torch
print(f"Torch Version: {torch.__version__}")

import transformers
print(f"transformers (Adapter) Version: {transformers.__version__}")

Torch Version: 1.8.1
transformers (Adapter) Version: 2.0.1


In [4]:
dateStamp = '0814_v3'

In [5]:
from transformers import RobertaTokenizer
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

In [6]:
from ner_dataset import get_trainset_data_loader
data_path = "./NER_multilabel_data_v3.csv"
#還需要改資料裡面的
all_tags, trainset, trainloader = get_trainset_data_loader(tokenizer, BATCH_SIZE=128, data_path = data_path)

labels: ['B-art' 'B-eve' 'B-geo' 'B-gpe' 'B-nat' 'B-org' 'B-per' 'B-tim'
 'CountryCode' 'CryptoCurrencyCode' 'CurrencyCode' 'Event' 'Float' 'I-art'
 'I-eve' 'I-geo' 'I-gpe' 'I-nat' 'I-org' 'I-per' 'I-tim' 'Integer'
 'Location' 'Month' 'O' 'Object' 'Party' 'Race' 'SpecialTerm'
 'TemporalUnit' 'Time' 'Timezone' 'US_States']


In [7]:
print(f"Example Training Sentence: {trainset.sentences[0]}")

Example Training Sentence: ['Thousands', 'Ġof', 'Ġdemon', 'str', 'ators', 'Ġhave', 'Ġmar', 'ched', 'Ġthrough', 'ĠLondon', 'Ġto', 'Ġpro', 'test', 'Ġthe', 'Ġwar', 'Ġin', 'ĠIraq', 'Ġand', 'Ġdemand', 'Ġthe', 'Ġwith', 'draw', 'al', 'Ġof', 'ĠBritish', 'Ġtro', 'ops', 'Ġfrom', 'Ġthat', 'Ġcountry', '.']


In [8]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=len(all_tags),
    label2id = trainset.label_map, 
    id2label = trainset.id2label
)

name = model.load_adapter("./save_adapters/ALL_tag_0730")
model.add_tagging_head(
        name,
        num_labels=len(trainset.label_map.keys()), overwrite_ok=True
      )
model.train_adapter(name)

In [9]:
device_id = 0
device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")

In [10]:
all_tags = ['O','B-art','I-org','B-geo','Race','I-gpe','I-tim','Object','Integer','I-per','TemporalUnit','B-org','CountryCode','B-gpe','B-eve','Party','I-geo','I-art','CryptoCurrencyCode','I-nat','Event','B-tim','Time','SpecialTerm','CurrencyCode','I-eve','Float','Month','B-per','Location','Timezone','US_States','B-nat']

In [ ]:
for index, tag in enumerate(all_tags):
    if index % 2 == device_id:
        print(f"\nSkip {tag}.\n")
        continue
    model = RobertaModelWithHeads.from_pretrained(
        "roberta-base",
        config=config,
        )


    try:
        model.add_adapter(tag)
        model.add_tagging_head(
            tag,
            num_labels=1
          )
    except: pass
    model.train_adapter(tag)
    model = model.to(device)
    
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
                    {
                        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                        "weight_decay": 1e-5,
                    },
                    {
                        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                        "weight_decay": 0.0,
                    },
                ]
    optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=1e-4)
    
    for epoch in range(4):
        print(f"\n{tag}: epoch {epoch}")
        for i, data in enumerate(trainloader):

            tokens_tensors, segments_tensors, \
            masks_tensors, labels = [t.to(device) for t in data]

            outputs = model(input_ids = tokens_tensors,
                attention_mask=masks_tensors,
                token_type_ids=segments_tensors)


            logits = outputs[0]

            current_label = labels.view(-1, labels.shape[-1])[:, trainset.label_map[tag]]
            current_label = current_label.view(-1)

            active_logits = logits.view(-1, logits.shape[-1])[masks_tensors.view(-1) == 1]
            active_labels = current_label[masks_tensors.view(-1)== 1]

            actual = current_label[masks_tensors.view(-1)== 1].float().view(-1,1)
            """

            actual = torch.ones(active_logits.shape, device = device)

            actual[:, 0] = (active_labels == 0).long()
            actual[:, 1] = (active_labels == 1).long()"""


            loss_fct = torch.nn.BCEWithLogitsLoss()

            loss = loss_fct(active_logits, actual)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            if i % 100 == 0:
                print(f"\tLoss: {loss}")
        telegram_bot_sendtext(f"\n{tag}: epoch {epoch}, loss = {loss}")
        filename = f"{tag}_epoch_{epoch}_{dateStamp}"
        model.save_adapter(f"./save_adapters/{filename}", model.active_adapters[0])
        model.save_head(f"./save_heads/{filename}", model.active_head)
    filename = f"{tag}_{dateStamp}"
    model.save_adapter(f"./save_adapters/{filename}", model.active_adapters[0])
    model.save_head(f"./save_heads/{filename}", model.active_head)


Skip O.



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere


B-art: epoch 0
	Loss: 0.5501952767372131


In [ ]:
label_id_mapping = trainset.label_map

id_label_mapping = dict()
for key in label_id_mapping.keys():
    id_label_mapping[label_id_mapping[key]] = key

def test_model(model, sentence, device = "cpu"):
    tokenized_sentence = torch.tensor([tokenizer.encode(sentence)])
    pos = torch.tensor([[0] * len(tokenized_sentence)])
    tags = torch.tensor([[1] * len(tokenized_sentence)])

    model = model.to(device)
    outputs = model(input_ids=tokenized_sentence.to(device), 
                    token_type_ids=pos.to(device), 
                    attention_mask=tags.to(device))

    logits = outputs[0]

    _, pred_labels = torch.max(logits, 2)

    out_labels = []
    for row in pred_labels:
        result = list(map(lambda x: id_label_mapping[int(x)], row))
        out_labels.append(result)
    #return tokenizer.tokenize(sentence), out_labels[0], logits
    return tokenizer.tokenize(sentence), out_labels[0][1:-1], logits[:, 1:-1]

In [ ]:
sentence = "Dan will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."

In [ ]:
tokenized_sentence = torch.tensor([tokenizer.encode(sentence)])
pos = torch.tensor([[0] * len(tokenized_sentence)])
tags = torch.tensor([[1] * len(tokenized_sentence)])

model = model.to(device)
outputs = model(input_ids=tokenized_sentence.to(device), 
                token_type_ids=pos.to(device), 
                attention_mask=tags.to(device))

In [ ]:
for i, text in enumerate(tokenizer.tokenize(sentence)):
    print(f"{text}: {outputs[0].view(-1)[i]}")

In [ ]:
sentence = "Dan Will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."
sen, pred, logits = test_model(model, sentence, device = 'cpu')

In [ ]:
a = tokenizer.tokenize(sentence)[1]

In [ ]:
np.array(sen)

In [ ]:
np.array(pred)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def interact_word(i):
    print(i)
    print(sen[i])
    target = out[i]

    for i in range(len(target)):
        print(f"{i} {id_label_mapping[i].ljust(6)} \t: {target[i]:.5f}")

In [ ]:
out = logits[0]
interact(lambda x: interact_word(x), x=widgets.IntSlider(min=0, max=len(sen)-1, step=1, value=0))

In [ ]:
print("OK")